In [ ]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from scipy.stats import uniform

# 스케일링
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# 모델링
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, validation_curve
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

# 평가
from sklearn.metrics import mean_squared_error, root_mean_squared_error, mean_absolute_error, r2_score

# 시각화
from sklearn.tree import plot_tree
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
import matplotlib.font_manager as fm
import matplotlib

font_path = 'C:\\Windows\\Fonts\\batang.ttc'
font = fm.FontProperties(fname=font_path).get_name()
matplotlib.rc('font', family=font)

# 데이터 로드

In [4]:
pd.set_option('display.max_rows', None)           # DataFrame/Series 출력 시 모든 행 보기
pd.set_option('display.max_seq_items', None) 

In [5]:
fin_df_1 = pd.read_csv('./data/dust2/subway1_weather_dust_clean.csv', encoding = 'utf-8')  # 피쳐 수 14개 데이터
fin_df_1 = fin_df_1.drop(['Unnamed: 0', 'date'], axis=1)
print(fin_df_1.columns)
display(fin_df_1.head())

Index(['ride_count', 'AvgTemp(°C)', 'Rainfall(mm)', 'WindSpeed(m/s)',
       'Humidity(%)', 'isolation(MJ/m2)', 'holiday_flag', '미세 먼지 농도', 'PM10',
       'PM2.5', '오 존', '이산화질소', '일산화탄소', '아황산가스'],
      dtype='object')


,ride_count,AvgTemp(°C),Rainfall(mm),WindSpeed(m/s),Humidity(%),isolation(MJ/m2),holiday_flag,미세 먼지 농도,PM10,PM2.5,오 존,이산화질소,일산화탄소,아황산가스
0,39420.0,-5.0,0.0,2.1,49.5,7.84,1,25.0,36.0,25.0,0.02,0.03,0.6,0.003
1,11807.0,-5.0,0.0,2.1,49.5,7.84,1,25.0,36.0,25.0,0.02,0.03,0.6,0.003
2,20944.0,-5.0,0.0,2.1,49.5,7.84,1,25.0,36.0,25.0,0.02,0.03,0.6,0.003
3,17798.0,-5.0,0.0,2.1,49.5,7.84,1,25.0,36.0,25.0,0.02,0.03,0.6,0.003
4,13578.0,-5.0,0.0,2.1,49.5,7.84,1,25.0,36.0,25.0,0.02,0.03,0.6,0.003


# EDA

In [6]:
corr_matrix_1 = fin_df_1.corr(method='spearman', numeric_only=True)   # 스피어만 상관계수
corr_matrix_1

,ride_count,AvgTemp(°C),Rainfall(mm),WindSpeed(m/s),Humidity(%),isolation(MJ/m2),holiday_flag,미세 먼지 농도,PM10,PM2.5,오 존,이산화질소,일산화탄소,아황산가스
ride_count,1.000000,0.001652,-0.052207,-0.061108,-0.065588,0.031462,-0.335814,0.025001,0.028220,0.024928,-0.011446,0.120991,0.056941,0.051023
AvgTemp(°C),0.001652,1.000000,0.177543,-0.101151,0.426665,0.348323,0.007635,0.061422,-0.355919,-0.264573,0.450517,-0.330905,-0.439979,-0.255282
Rainfall(mm),-0.052207,0.177543,1.000000,0.191301,0.602271,-0.473081,0.024507,0.027963,-0.273287,-0.216493,-0.115302,-0.144468,-0.163256,-0.272984
WindSpeed(m/s),-0.061108,-0.101151,0.191301,1.000000,-0.041119,0.019145,-0.022540,-0.103053,-0.191162,-0.302266,0.206470,-0.479980,-0.411116,-0.256509
Humidity(%),-0.065588,0.426665,0.602271,-0.041119,1.000000,-0.427999,0.018756,0.044102,-0.302385,-0.140750,-0.086390,-0.192020,-0.153340,-0.317197
isolation(MJ/m2),0.031462,0.348323,-0.473081,0.019145,-0.427999,1.000000,0.002004,-0.038728,0.030937,-0.025858,0.607564,-0.177455,-0.236507,0.007186
holiday_flag,-0.335814,0.007635,0.024507,-0.022540,0.018756,0.002004,1.000000,0.021390,-0.044020,-0.018855,0.110058,-0.204814,-0.081754,-0.074958
미세 먼지 농도,0.025001,0.061422,0.027963,-0.103053,0.044102,-0.038728,0.021390,1.000000,-0.047505,-0.034927,-0.062525,0.060137,0.036263,0.007461
PM10,0.028220,-0.355919,-0.273287,-0.191162,-0.302385,0.030937,-0.044020,-0.047505,1.000000,0.888479,-0.049505,0.589536,0.689631,0.536264
PM2.5,0.024928,-0.264573,-0.216493,-0.302266,-0.140750,-0.025858,-0.018855,-0.034927,0.888479,1.000000,-0.050944,0.590786,0.728884,0.528595


In [7]:
# 사분위 수 범위 - 이상치 제거 
import pandas as pd

series = pd.Series(fin_df_1['ride_count'])
description = series.describe()

q1 = description['25%']
q2 = description['50%']
q3 = description['75%']

print(f"1사분위수 (Q1): {q1}")
print(f"2사분위수 (Q2, 중앙값): {q2}")
print(f"3사분위수 (Q3): {q3}")

iqr = q3 - q1
min_q = q1 - 1.5 * iqr
max_q = q3 + 1.5 * iqr

print(f'min : {min_q}, max : {max_q}')
print(fin_df_1['ride_count'].max())
print(fin_df_1['ride_count'].min())

1사분위수 (Q1): 12176.0
2사분위수 (Q2, 중앙값): 18664.5
3사분위수 (Q3): 27556.75
min : -10895.125, max : 50627.875
78390.0
2613.0


In [10]:
fin_df_1 = pd.read_csv('./data/dust2/subway1_weather_dust_clean.csv', encoding = 'utf-8')  # 피쳐 수 14개 데이터
fin_df_1 = fin_df_1.drop(['Unnamed: 0', 'date'], axis=1)
fin_df_2 = pd.read_csv('./data/dust2/subway2_weather_dust_clean.csv', encoding = 'utf-8')  # 피쳐 수 14개 데이터
fin_df_2 = fin_df_2.drop(['Unnamed: 0', 'date'], axis=1)
fin_df_3 = pd.read_csv('./data/dust2/subway3_weather_dust_clean.csv', encoding = 'utf-8')  # 피쳐 수 14개 데이터
fin_df_3 = fin_df_3.drop(['Unnamed: 0', 'date'], axis=1)
fin_df_4 = pd.read_csv('./data/dust2/subway4_weather_dust_clean.csv', encoding = 'utf-8')  # 피쳐 수 14개 데이터
fin_df_4 = fin_df_4.drop(['Unnamed: 0', 'date'], axis=1)
fin_df_5 = pd.read_csv('./data/dust2/subway5_weather_dust_clean.csv', encoding = 'utf-8')  # 피쳐 수 14개 데이터
fin_df_5 = fin_df_5.drop(['Unnamed: 0', 'date'], axis=1)
fin_df_6 = pd.read_csv('./data/dust2/subway6_weather_dust_clean.csv', encoding = 'utf-8')  # 피쳐 수 14개 데이터
fin_df_6 = fin_df_6.drop(['Unnamed: 0', 'date'], axis=1)
fin_df_7 = pd.read_csv('./data/dust2/subway7_weather_dust_clean.csv', encoding = 'utf-8')  # 피쳐 수 14개 데이터
fin_df_7 = fin_df_7.drop(['Unnamed: 0', 'date'], axis=1)
fin_df_8 = pd.read_csv('./data/dust2/subway8_weather_dust_clean.csv', encoding = 'utf-8')  # 피쳐 수 14개 데이터
fin_df_8 = fin_df_8.drop(['Unnamed: 0', 'date'], axis=1)
fin_df_9 = pd.read_csv('./data/dust2/subway9_weather_dust_clean.csv', encoding = 'utf-8')  # 피쳐 수 14개 데이터
fin_df_9 = fin_df_9.drop(['Unnamed: 0', 'date'], axis=1)


-----------

In [ ]:
# 데이터 분할
X_1 = fin_df_1.drop('ride_count', axis=1)
y_1 = fin_df_1['ride_count']


# 스케일링
scaler = StandardScaler()
X_scaled_1 = scaler.fit_transform(X_1)

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_scaled_1, y_1, random_state=0)

--------

# Gradient_descent

In [ ]:
# 데이터 준비
X_data = df_1.drop('ride_count', axis=1)
Y_data = df_1['ride_count']

X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, random_state=42)

# 정규화
x_scaler = StandardScaler()
X_train_scaled = x_scaler.fit_transform(X_train)
X_test_scaled = x_scaler.transform(X_test)

y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1, 1))

# 미니배치 경사하강법 함수
def mini_batch_gradient_descent(X, y, batch_size=32, learning_rate=0.01, max_iter=1000):
    w0 = np.zeros((1, 1))
    w1 = np.zeros((X.shape[1], 1))
    N = len(X)

    for epoch in range(max_iter):
        indices = np.random.permutation(N)
        X_shuffled = X[indices]
        y_shuffled = y[indices]

        for i in range(0, N, batch_size):
            X_batch = X_shuffled[i:i+batch_size]
            y_batch = y_shuffled[i:i+batch_size]
            b = len(X_batch)

            y_pred = w0 + np.dot(X_batch, w1)
            diff = y_batch - y_pred

            w0_diff = -learning_rate * 2/b * np.sum(diff)
            w1_diff = -learning_rate * 2/b * np.dot(X_batch.T, diff)

            w0 -= w0_diff
            w1 -= w1_diff

    return w0, w1

# 수동 Randomized SearchCV
import random

param_grid = {
    'learning_rate': [0.001, 0.005, 0.01, 0.05, 0.1],
    'batch_size': [8, 16, 32, 64],
    'max_iter': [300, 500, 1000, 2000]
}

n_iter_search = 10
kf = KFold(n_splits=3, shuffle=True, random_state=42)
best_score = float('inf')
best_params = None

for _ in range(n_iter_search):
    params = {
        'learning_rate': random.choice(param_grid['learning_rate']),
        'batch_size': random.choice(param_grid['batch_size']),
        'max_iter': random.choice(param_grid['max_iter'])
    }

    cv_scores = []

    for train_idx, val_idx in kf.split(X_train_scaled):
        X_cv_train, X_cv_val = X_train_scaled[train_idx], X_train_scaled[val_idx]
        y_cv_train, y_cv_val = y_train_scaled[train_idx], y_train_scaled[val_idx]

        w0, w1 = mini_batch_gradient_descent(
            X_cv_train, y_cv_train,
            learning_rate=params['learning_rate'],
            batch_size=params['batch_size'],
            max_iter=params['max_iter']
        )

        y_val_pred = w0 + np.dot(X_cv_val, w1)
        score = mean_squared_error(y_cv_val, y_val_pred)
        cv_scores.append(score)

    mean_score = np.mean(cv_scores)

    if mean_score < best_score:
        best_score = mean_score
        best_params = params

print(f"Best Params: {best_params}, CV MSE: {best_score:.4f}")

# 최적 파라미터로 다시 학습
w0, w1 = mini_batch_gradient_descent(
    X_train_scaled, y_train_scaled,
    learning_rate=best_params['learning_rate'],
    batch_size=best_params['batch_size'],
    max_iter=best_params['max_iter']
)

# 테스트셋 예측 및 평가
y_test_pred_scaled = w0 + np.dot(X_test_scaled, w1)
y_test_pred = y_scaler.inverse_transform(y_test_pred_scaled)

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
def evaluate_regression(y_true, y_pred): 
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f'MSE : {mse:.2f}, RMSE : {rmse:.2f}, MAE : {mae:.2f}, R2 : {r2:.4f}')

evaluate_regression(y_test.values, y_test_pred.ravel())

--------

# Linear Regression

In [ ]:
# 데이터 불러오기
df_1 = pd.read_csv('./data/subway1_weather_dust_clean.csv')
df_1 = df_1.drop(columns=['Unnamed: 0', 'date'])

# 입력-라벨 분리
X_data = df_1.drop('ride_count', axis=1)
Y_data = df_1['ride_count']

# 학습/테스트 분리
X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, random_state=42)

# 다항 특성 생성
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

# 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_poly)
X_test_scaled = scaler.transform(X_test_poly)

# 모델 훈련
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)

# 예측
y_pred = lr.predict(X_test_scaled)

# 평가 함수
def evaluate_regression(y_true, y_pred): 
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    print(f'MSE : {mse:.2f}, RMSE : {rmse:.2f}, MAE : {mae:.2f}, R2 : {r2:.4f}')

# 모델 평가
evaluate_regression(y_test, y_pred)

-----

# SVM

In [ ]:
# 정규화
x_scaler = StandardScaler()
X_train_scaled = x_scaler.fit_transform(X_train)
X_test_scaled = x_scaler.transform(X_test)

# 하이퍼파라미터 분포 정의 (무작위 탐색용)
param_distributions = {
    'C': uniform(0.1, 100),        # 0.1 ~ 100 사이 연속값
    'epsilon': uniform(0.1, 10),   # 0.1 ~ 10 사이 연속값
    'gamma': ['scale', 0.01, 0.1, 1]
}

# SVR 모델 + RandomizedSearchCV
random_search = RandomizedSearchCV(
    SVR(kernel='rbf'),
    param_distributions=param_distributions,
    n_iter=20,  # 시도할 파라미터 조합 수
    cv=5,
    scoring='r2',
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train_scaled, y_train)

# 최적 모델
best_svr = random_search.best_estimator_
print("Best Parameters:", random_search.best_params_)

# 테스트셋 예측
y_pred = best_svr.predict(X_test_scaled)

# 평가 함수
def evaluate_regression(y_true, y_pred): 
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f'MSE : {mse:.2f}, RMSE : {rmse:.2f}, MAE : {mae:.2f}, R2 : {r2:.4f}')

# 평가 실행
evaluate_regression(y_test, y_pred)

-------

# 규제선형모델

### Ridge

In [ ]:
# 모델링
reg = Ridge(alpha=1.0)
reg.fit(X_train_1, y_train_1)

# 모델 평가 
print(reg.score(X_train_1, y_train_1), reg.score(X_test_1, y_test_1))

0.1023464190025054 0.08592258329267


In [27]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import Ridge
from scipy.stats import uniform

params = {'alpha': uniform(0.01, 100)}
ridge = Ridge()
rand_search = RandomizedSearchCV(ridge, param_distributions=params, n_iter=10, cv=5, random_state=42)
rand_search.fit(X_train_1, y_train_1)

print("Best alpha:", rand_search.best_params_)
print("Best score:", rand_search.best_score_)

Best alpha: {'alpha': np.float64(95.08143064099163)}
Best score: 0.10076443519221637


In [12]:
# 데이터 분할
X_1 = fin_df_1.drop('ride_count', axis=1)
y_1 = fin_df_1['ride_count']


# 스케일링
scaler = StandardScaler()
X_scaled_1 = scaler.fit_transform(X_1)

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_scaled_1, y_1, random_state=0)

# 모델링
reg = Ridge(alpha=95.08)
reg.fit(X_train_1, y_train_1)

# 모델 평가 
print(reg.score(X_train_1, y_train_1), reg.score(X_test_1, y_test_1))

0.10234002093909678 0.0860196374003881


In [ ]:
# 모든 노선 DataFrame과 이름을 리스트로 관리
line_df = [
    ('1호선', fin_df_1),
    ('2호선', fin_df_2),
    ('3호선', fin_df_3),
    ('4호선', fin_df_4),
    ('5호선', fin_df_5),
    ('6호선', fin_df_6),
    ('7호선', fin_df_7),
    ('8호선', fin_df_8),
    ('9호선', fin_df_9),
]

# 결과 저장용
results = []

# 반복문으로 분석
for line_name, df in line_df:
    print(f"{line_name} 분석 중")

     # 독립변수 / 종속변수 분리
    X = df.drop(columns=['ride_count'], axis=1)
    y = df['ride_count']

    # 스케일링
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # 데이터 분리
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    # LassoCV 모델 학습
    model = Lasso(alpha=10, max_iter=10000)
    model.fit(X_train, y_train)

    # 예측 및 평가
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = root_mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # 결과 저장
    results.append({
        '노선': line_name,
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R2' : r2
        # '사용된 변수 개수': nonzero
    })

# 결과 정리
results_df = pd.DataFrame(results)

print("\n Lasso 회귀 결과 요약:")
print(f'MSE: { mse }, RMSE: { rmse }, MAE: { mae }, R2: { r2 }')
print(results_df)



1호선 분석 중
2호선 분석 중
3호선 분석 중
4호선 분석 중
5호선 분석 중
6호선 분석 중
7호선 분석 중
8호선 분석 중
9호선 분석 중

 Lasso 회귀 결과 요약:
MSE: 63972695.09087698, RMSE: 7998.293261119961, MAE: 6249.8160950759375, R2: 0.08787259579250606
    노선           MSE          RMSE           MAE        R2
0  1호선  1.495360e+08  12228.490523   9155.067881  0.105769
1  2호선  3.881251e+08  19700.889645  14949.169321  0.076053
2  3호선  1.234341e+08  11110.091059   8381.303436  0.066566
3  4호선  8.847390e+07   9406.056682   7471.495355  0.121195
4  5호선  4.686444e+07   6845.760651   5317.530218  0.121449
5  6호선  1.902467e+07   4361.728219   3443.821182  0.132118
6  7호선  4.047525e+07   6362.015955   4338.832989  0.189195
7  8호선  2.523340e+07   5023.285637   4088.112843  0.181391
8  9호선  6.397270e+07   7998.293261   6249.816095  0.087873


### Lasso

In [13]:
# 데이터 분할
X = fin_df_1.drop('ride_count', axis=1)
y = fin_df_1['ride_count']

# 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state=42)

# 모델링
las = Lasso(alpha=0.01)
las.fit(X_train, y_train)

# 모델 평가 
print(las.score(X_train, y_train), las.score(X_test, y_test))

0.09709459269314347 0.10227713654329951


In [14]:
# 하이퍼파라미터 최적화 
from sklearn.linear_model import LassoCV

las_cv = LassoCV(alphas=[0.01, 0.1, 1, 10])
las_cv.fit(X_train, y_train)

print("Best alpha:", las_cv.alpha_)
print("Test score:", las_cv.score(X_test, y_test))

Best alpha: 0.1
Test score: 0.10227704365443013


In [15]:
# 데이터 분할
X = fin_df_1.drop('ride_count', axis=1)
y = fin_df_1['ride_count']

# 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state=42)

# 모델링
las = Lasso(alpha=0.102)
las.fit(X_train, y_train)

# 모델 평가 
print(las.score(X_train, y_train), las.score(X_test, y_test))

0.09709459004743515 0.10227704153517514


------

# DecisionTree

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_1, y_1, random_state= 0)

sc = StandardScaler()
sc_y = StandardScaler()
x_train_scaled = sc.fit_transform(X_train)
x_test_scaled = sc.transform(X_test)

y_train_scaled = sc_y.fit_transform(y_train.to_frame())  # 결과는 (n_samples, 1)
y_test_scaled = sc_y.transform(y_test.to_frame())

dummy = DummyRegressor(strategy="mean")
dummy.fit(x_train_scaled, y_train_scaled)
print("Baseline R2:", dummy.score(x_train_scaled, y_train_scaled))

X_small, y_small = x_train_scaled[:100], y_train_scaled[:100]
model.fit(X_small, y_small)
print("R2 on that tiny subset:", model.score(X_small, y_small))

dt = DecisionTreeRegressor(random_state=42)
param_grid = {
    "max_depth": [None] + list(range(3, 16)),
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": [None, "sqrt", "log2"],
}

gs = GridSearchCV(
    DecisionTreeRegressor(random_state=42),
    param_grid,
    cv=5,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1,
    verbose=1
)
gs.fit(x_train_scaled, y_train_scaled)

print("Best params:", gs.best_params_)

# 3. 최적 모델로 테스트 셋 평가
best_dt = gs.best_estimator_
y_pred = best_dt.predict(x_train)
r2_train = r2_score(y_train,y_pred)
mse_train = mean_squared_error(y_train,y_pred)
print('r2 score:',r2_train)
print('MSE:',mse_train)

best_dt.score(x_train_scaled,y_train_scaled)

# 평가
mse_train = mean_squared_error(y_train, pred_train)
r2_train = r2_score(y_train,pred_train)
mse_test = mean_squared_error(y_test,pred_test)
r2_test = r2_score(y_test, pred_test)

print(f'훈련 데이터에 대한 평가 : mse : {mse_train}, r2 : {r2_train}')
print(f'테스트 데이터에 대한 평가 : mse : {mse_test}, r2 : {r2_test}')

-------------------

# 랜덤포레스트 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_1, y_1, random_state=0)
print(X_train.shape, X_test.shape)

# 하이퍼 파라미터 튜닝
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

rf = RandomForestRegressor(random_state=0)

grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train, y_train)
print("최적 하이퍼파라미터:", grid_search.best_params_)
best_params = grid_search.best_params_

# 모델 생성
rf_best = RandomForestRegressor(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    random_state=0
)

# 학습
rf_best.fit(X_train, y_train)

# 평가
print(rf_best.score(X_train, y_train), rf_best.score(X_test, y_test))

-----------

### XGBOOST

In [17]:
!pip install xgboost

In [18]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [19]:
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y_1, random_state=0)

In [ ]:
# 데이터 분할
X_1 = fin_df_1.drop('ride_count', axis=1)
y_1 = fin_df_1['ride_count'] # x 스케일링 값
y_2 = fin_df_1[['ride_count']]

# 스케일링 - x
scaler = StandardScaler()
X_scaled_1 = scaler.fit_transform(X_1)
y_scaled_1 = scaler.fit_transform(y_1)

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_scaled_1, y_1, random_state=0)

# 스케일링 - y
scaler = StandardScaler()
X_scaled_1 = scaler.fit_transform(X_1)
y_scaled_1 = scaler.fit_transform(y_2)

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_scaled_1, y_scaled_1, random_state=0)

In [ ]:
# 스케일링 - y
xgb_clf = XGBRegressor(
    n_estimators=100,
    max_depth=3,
    learning_rate=0.01,
    random_state=0,
    colsample_bytree=1.0, 
    subsample=0.8
)

xgb_clf.fit(X_train_1, y_train_1)

y_pred_train = xgb_clf.predict(X_train_1)
y_pred_test = xgb_clf.predict(X_test_1)

y_pred_train_inv = scaler.inverse_transform(y_pred_train.reshape(-1, 1))
y_pred_test_inv = scaler.inverse_transform(y_pred_test.reshape(-1, 1))

y_train_inv = scaler.inverse_transform(y_train_1.reshape(-1, 1))
y_test_inv = scaler.inverse_transform(y_test_1.reshape(-1, 1))


# 평가
print("Train R2:", r2_score(y_train_inv, y_pred_train_inv))
print("Test R2:", r2_score(y_test_inv, y_pred_test_inv))

Train R2: 0.09280209264903405
Test R2: 0.07837734775466576


In [ ]:
# 스케일링 - x
xgb_clf = XGBRegressor(
    n_estimators=100,
    max_depth=3,
    learning_rate=0.1,
    random_state=0,
    colsample_bytree=0.8, 
    subsample=1.0
)

xgb_clf.fit(X_train_1, y_train_1)

y_pred_train = xgb_clf.predict(X_train_1)
y_pred_test = xgb_clf.predict(X_test_1)

# 평가
print("Train R2:", r2_score(y_train_1, y_pred_train))
print("Test R2:", r2_score(y_test_1, y_pred_test))

In [ ]:
# 하이퍼파라미터 최적화 
xgb = XGBRegressor(random_state=0)

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.3],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

grid_search = GridSearchCV(
    estimator=xgb_clf,
    param_grid=param_grid,
    cv=3,
    scoring='neg_mean_squared_error',  # 회귀 문제일 때
    verbose=1,
    n_jobs=-1
)

grid_search.fit(X_train_1, y_train_1)

print("Best params:", grid_search.best_params_)
print("Best score (neg MSE):", grid_search.best_score_)

Fitting 3 folds for each of 72 candidates, totalling 216 fits
Best params: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}
Best score (neg MSE): -0.8900042830886973


### LightGBM

In [ ]:
# 데이터 불러오기
df_rf_1 = pd.read_csv('./data/미세먼지+지하철+날씨 (2)/subway1_weather_dust_clean.csv')
df_rf_1 = df_rf_1.drop(['Unnamed: 0', 'date'], axis=1)

from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score

# 데이터 분할
X1 = fin_df_1.drop(['ride_count'], axis=1)
y1 = fin_df_1['ride_count']

X_train, X_test, y_train, y_test = train_test_split(X1, y1, random_state=0)

# 검증용 데이터 분할
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, random_state=0, test_size=0.1)


# 하이퍼파라미터 튜닝
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

grid_search = GridSearchCV(
    estimator=LGBMRegressor(random_state=0),
    param_grid=param_grid,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train, y_train)
print("최적 하이퍼파라미터:", grid_search.best_params_)

# 최적 모델로 재학습
best_params = grid_search.best_params_

best_lgb = LGBMRegressor(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    random_state=0
)

# 학습 + 평가 데이터 지정
eval_set = [(X_tr, y_tr), (X_val, y_val)]
best_lgb.fit(X_tr, y_tr, eval_set=eval_set, eval_metric='rmse')

print("Train R2:", best_lgb.score(X_train, y_train))
print("Train R2:", best_lgb.score(X_test, y_test))